# Testing EUGENE `train` module

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing the training of EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

# Set-up

In [12]:
import numpy as np
import pandas as pd

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [14]:
import eugene as eu

# Benchmark params

In [127]:
BATCH_SIZE = 32
NUM_SEQS = 1000
SEQ_LEN = 66
OUT_DIMS = 2
NUM_WORKERS = 0

In [128]:
CNN_KWARGS=dict(channels=[4, 16, 32], 
                conv_kernels=[15, 5], 
                pool_kernels=[1, 1])
RNN_KWARGS=dict(output_dim=32,
                bidirectional=True,
                batch_first=True)
FCN_KWARGS=dict(hidden_dims=[50])

In [129]:
MODEL = "hybrid"
STRAND = "ss"
TASK = "regression"
LOSS_FXN = "mse"

# Instantiate model

In [130]:
eugene = eu.models.Hybrid(
    input_len=SEQ_LEN, 
    output_dim=OUT_DIMS,
    strand=STRAND,
    task=TASK,
    loss_fxn=LOSS_FXN,
    conv_kwargs=CNN_KWARGS,
    rnn_kwargs=RNN_KWARGS,
    fc_kwargs=FCN_KWARGS
)
eu.models.base.init_weights(eugene)
eugene

Hybrid(
  (hp_metric): R2Score()
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU()
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (4): ReLU()
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (recurrentnet): BasicRecurrent(
    (module): LSTM(32, 32, batch_first=True, bidirectional=True)
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=64, out_features=50, bias=True)
      (1): ReLU()
      (2): Linear(in_features=50, out_features=2, bias=True)
    )
  )
)

# Loss function

In [131]:
eugene.loss_fxn

<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Union[bool, NoneType] = None, reduce: Union[bool, NoneType] = None, reduction: str = 'mean') -> torch.Tensor>

In [132]:
import torch

In [133]:
# Length of strand
x_len = 66

# Substitute x and x_rev for their respective data
x = torch.randn(10, 4, x_len)
x_rev = torch.randn(10, 4, x_len)
outs = eugene(x, x_rev).squeeze(dim=1)
y = torch.randn(10, 2)
#y = torch.nn.functional.one_hot(torch.tensor(np.arange(10))).float()
outs.shape, y.shape

(torch.Size([10, 2]), torch.Size([10, 2]))

In [134]:
eugene.loss_fxn(outs, y)

tensor(1.0864, grad_fn=<MseLossBackward0>)

# Metrics

In [135]:
eugene.hp_metric, eugene.hp_metric_name

(R2Score(), 'r2')

In [136]:
eugene.hp_metric.num_outputs

2

In [137]:
eugene._calculate_metric(outs, y.long())

tensor(-0.0458, grad_fn=<SqueezeBackward0>)

# Load data

In [138]:
sdata = eu.datasets.random1000(binary=False)
sdata

Kept 1000 sequences with targets, dropped 0 sequences with no targets


SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = None
ohe_rev_seqs = None
seqs_annot: 'TARGETS'
pos_annot: PyRanges object with 1400 features
seqsm: None
uns: None

In [139]:
eu.pp.prepare_data(sdata)

  0%|          | 0/3 [00:00<?, ?it/s]

reverse_complement
one_hot_encode
train_test_split
SeqData object modified:
	rev_seqs: None -> 1000 rev_seqs added
	ohe_seqs: None -> 1000 ohe_seqs added
	ohe_rev_seqs: None -> 1000 ohe_rev_seqs added
    seqs_annot:
        + TRAIN


In [140]:
sdata.seqs_annot["TARGETS_2"] = sdata.seqs_annot["TARGETS"]

In [141]:
sdataset = sdata.to_dataset(label=["TARGETS", "TARGETS_2"])

No transforms given, assuming just need to tensorize).


In [142]:
sdataset[0]

(tensor([115., 101., 113.,  48.,  48.,  48.]),
 tensor([[1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
        

# Train (and time) with PyTorch Lightning

In [143]:
from eugene.train import fit

In [145]:
LOG_DIR = "/cellar/users/aklie/projects/EUGENE/tests/_logs/"
OUT_DIR = "/cellar/users/aklie/projects/EUGENE/tests/_out/"

In [147]:
fit(eugene, sdata=sdata, gpus=1, epochs=50, num_workers=4, target_label=["TARGETS", "TARGETS_2"], log_dir=LOG_DIR, out_dir=OUT_DIR, batch_size=128)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


No transforms given, assuming just need to tensorize).
No transforms given, assuming just need to tensorize).


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name         | Type                      | Params
-----------------------------------------------------------
0 | hp_metric    | R2Score                   | 0     
1 | convnet      | BasicConv1D               | 3.6 K 
2 | recurrentnet | BasicRecurrent            | 16.9 K
3 | fcnet        | BasicFullyConnectedModule | 3.4 K 
-----------------------------------------------------------
23.8 K    Trainable params
0         Non-trainable params
23.8 K    Total params
0.095     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 13
/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Predicting: 0it [00:00, ?it/s]

ValueError: all input arrays must have the same shape

In [172]:
sdata.seqs_annot

,TARGETS,TRAIN,PREDICTIONS
seq000,0.866168,True,-0.690370
seq001,0.800737,True,-0.636537
seq002,0.703108,True,-0.671358
seq003,0.465782,False,-0.590072
seq004,0.676781,False,-0.653740
...,...,...,...
seq995,0.001721,True,-0.652059
seq996,0.617911,True,-0.642589
seq997,0.565960,True,-0.611037
seq998,0.293470,True,-0.671817


In [9]:
!kill 1640074

In [11]:
%load_ext tensorboard
%env TENSORBOARD_BINARY /cellar/users/aklie/opt/miniconda3/envs/eugene/bin/tensorboard
%tensorboard --logdir /cellar/users/aklie/projects/EUGENE/tests/_logs/default/version_110

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
env: TENSORBOARD_BINARY=/cellar/users/aklie/opt/miniconda3/envs/eugene/bin/tensorboard


Reusing TensorBoard on port 6006 (pid 1640553), started 0:00:08 ago. (Use '!kill 1640553' to kill it.)

In [7]:
from IPython.display import display, HTML

In [ ]:
saved_t = pd.read_csv("../_out/train_predictions.tsv", index_col=0, sep="\t")
np.allclose(saved_t["PREDICTION"].values, sdata.seqs_annot.loc[saved_t.index]["PREDICTIONS"].values)


True

In [58]:
sdata.seqs_annot.merge(preds, left_index=True, right_index=True)

,TARGETS,TRAIN,0
seq001,1.0,False,0.6329051
seq002,0.0,True,0.49575517
seq003,1.0,False,0.546878
seq004,0.0,False,0.4899232
seq005,0.0,False,0.6157094
...,...,...,...
seq996,0.0,True,0.48639736
seq997,0.0,True,0.6196456
seq998,0.0,False,0.5316826
seq999,0.0,True,0.35639912


In [39]:
pd.DataFrame(np.concatenate(v, axis=0))

,0,1,2
0,seq001,0.6047737,1.0
1,seq003,0.5640073,0.0
2,seq004,0.5120066,1.0
3,seq005,0.5677963,0.0
4,seq008,0.58250546,0.0
...,...,...,...
595,seq991,0.48263985,1.0
596,seq992,0.61272997,0.0
597,seq994,0.55296373,1.0
598,seq998,0.5503518,1.0


In [ ]:
eu.train.fit(eugene, sdata=sdata, epochs=5, log_dir="../_logs") 

In [22]:
from pytorch_lightning import Trainer

In [23]:
trainer = Trainer(max_epochs=3)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [24]:
trainer.fit(eugene, sdataloader) 

/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name         | Type                      | Params
-----------------------------------------------------------
0 | convnet      | BasicConv1D               | 3.6 K 
1 | recurrentnet | BasicRecurrent            | 16.9 K
2 | fcnet        | BasicFullyConnectedModule | 3.3 K 
3 | r_squared    | R2Score                   | 0     
-----------------------------------------------------------
23.8 K    Trainable params
0         Non-trainable params
23.8 K    Total params
0.095     Total estimated model params size (MB)
/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, 

Training: 0it [00:00, ?it/s]

In [19]:
next(eugene.parameters())

Parameter containing:
tensor([[[-9.4785e-02, -1.7045e-03,  1.2167e-01, -1.2430e-01, -1.2435e-01,
           2.1259e-02, -7.5152e-02,  7.2229e-02,  1.0589e-01, -8.6553e-02,
          -1.1654e-01, -1.2605e-02,  1.0931e-01,  1.7115e-02, -8.7077e-03],
         [-7.3724e-02, -7.8527e-02, -5.5711e-02,  8.2650e-02, -1.0370e-01,
           1.4170e-01, -1.6458e-02,  9.5570e-03,  9.7584e-02,  1.2892e-01,
          -4.4364e-02, -1.1682e-02,  8.7450e-02, -1.9598e-02, -6.5818e-02],
         [-8.2107e-02, -3.5073e-02,  6.3604e-02,  8.6646e-02,  1.4163e-01,
          -4.8572e-02, -3.7498e-02, -7.4766e-02,  2.6753e-02, -9.3077e-02,
           1.2472e-01, -1.4999e-01,  7.3813e-02,  9.0117e-03,  7.4403e-02],
         [ 7.2231e-02,  1.1413e-01, -8.5916e-02, -7.6079e-02,  8.6604e-02,
           3.2138e-02,  6.6427e-02, -4.0387e-02,  1.5578e-01,  6.1688e-02,
           2.6640e-02, -5.1305e-02,  2.5741e-02, -1.4536e-01,  5.5284e-02]],

        [[-9.9758e-03,  2.3328e-02, -4.0814e-03, -3.5643e-03, -1.7920e-0

In [60]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [63]:
LOG_DIR = f"../_logs/batch_size-{BATCH_SIZE}.num_workers-{NUM_WORKERS}.num_seq-{NUM_SEQS}.seq_len-{SEQ_LEN}"
logger = TensorBoardLogger(LOG_DIR, name=MODEL, version=f"{STRAND}_{TASK}")
trainer = pl.Trainer(max_epochs=10, logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [64]:
trainer.fit(eugene, sdataloader)


  | Name         | Type                      | Params
-----------------------------------------------------------
0 | convnet      | BasicConv1D               | 3.6 K 
1 | recurrentnet | BasicRecurrent            | 16.9 K
2 | fcnet        | BasicFullyConnectedModule | 3.3 K 
3 | r_squared    | R2Score                   | 0     
-----------------------------------------------------------
23.8 K    Trainable params
0         Non-trainable params
23.8 K    Total params
0.095     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

---

# Scratch